In [ ]:
import numpy as np 
import torch
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from peft import load_peft_weights, set_peft_model_state_dict


In [31]:
import evaluate
bleu = evaluate.load("bleu")

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "SeaLLMs/SeaLLMs-v3-1.5B",
    load_in_4bit= False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None, 
)

In [27]:
prompt= "### Question:\n{}\n### Answer:\n{}"

data= pd.concat([pd.read_json('1k9_rlhf.json').rename(columns= {'answers': 'answer'}), pd.read_json('4k_rlhf.json')])
data= pd.DataFrame({'text': data.apply(lambda x: prompt.format(x['question'], ''), axis= 1), 'response': data.answer})

data= Dataset.from_pandas(data)
data= data.train_test_split(test_size=0.15, seed= 47)


# train_data= data['train']
eval_data= data['test']

In [28]:
eval_data_text= np.array_split(eval_data['text'], int(len(eval_data) / 16))

In [29]:
eval_data_text[0].tolist()

['### Question:\nNgân sách nhà nước có phân bổ cho quỹ bảo trì đường bộ hàng năm?\n### Answer:\n',
 '### Question:\nKính chắn gió của xe ô tô phải là loại kính gì?\n### Answer:\n',
 '### Question:\nCông việc bảo trì đường bộ gồm những gì?\n### Answer:\n',
 '### Question:\nXe mô tô có thể được kéo theo không?\n### Answer:\n',
 '### Question:\nNgười điều khiển xe máy chuyên dùng có cần giấy chứng nhận kiểm định không?\n### Answer:\n',
 '### Question:\nCác biện pháp bảo vệ kết cấu hạ tầng giao thông đường bộ bao gồm gì?\n### Answer:\n',
 '### Question:\nGiá vé tàu có phải do Ủy ban nhân dân tỉnh quyết định?\n### Answer:\n',
 '### Question:\nXe cơ giới phải có nguồn gốc như thế nào để được cấp đăng ký?\n### Answer:\n',
 '### Question:\nKhông nhường đường cho xe ưu tiên bị phạt bao nhiêu?\n### Answer:\n',
 '### Question:\nDự kiến từ ngày 01/7/2024, trẻ em trên 10 tuổi mới được ngồi ghế trước ô tô?\n### Answer:\n',
 '### Question:\nTừ ngày 15/8/2023 các trường hợp xe nhập khẩu chưa đăng ký s

In [16]:
config_generate= {
    "max_new_tokens": 512,
    "top_p": 0.95,
    "do_sample": True,
    'top_k': 100, 
    'temperature': 0.7
}


In [6]:
def chat(text):
  text= [prompt.format(i, "") for i in text]
  inputs= tokenizer.batch_encode_plus(text, max_length= 256, padding= "longest", truncation= True, return_tensors='pt').to("cuda")
  result= tokenizer.batch_decode(model.generate(input_ids= inputs['input_ids'], attention_mask= inputs['attention_mask'], 
                                **config_generate), skip_special_tokens= True)


  return result #[i.split('### Answer:')[1] for i in result]

In [34]:
# inference sft 
set_peft_model_state_dict(model, load_peft_weights('sft_model'))
# FastLanguageModel.for_inference(model)

result_sft= []
for i in eval_data: 
    result_sft.extend(chat(i.tolist()))

result_sft_response= [i.split('### Answer:')[1] for i in result_sft]

bleu.compute(predictions= result_sft_response, references= eval_data['response'])['bleu']

0.56


In [35]:
# inference ppo
set_peft_model_state_dict(model, load_peft_weights('ppo_model'))

result_ppo= []
for i in eval_data: 
    result_ppo.extend(chat(i.tolist()))
    
result_ppo_response= [i.split('### Answer:')[1] for i in result_ppo]

bleu.compute(predictions= result_ppo_response, references= eval_data['response'])['bleu']

0.8


In [ ]:
# test response score with reward 
from transformers import AutoTokenizer, pipeline
from peft import AutoPeftModelForSequenceClassification

In [ ]:
model_rm= AutoPeftModelForSequenceClassification.from_pretrained(
   "reward_model",
    num_labels=1
)
rm_tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct"
)

model_rm.config.update({"pad_token_id": rm_tokenizer.eos_token_id})
model_rm.to("cuda")

reward_pipe= pipeline(
    "sentiment-analysis",
    model= model_rm.merge_and_unload(),
    tokenizer= rm_tokenizer,
    return_token_type_ids= False,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

sent_kwargs = {"top_k": None, "function_to_apply": "none"}

In [41]:
# compute_score 
like= []

score_response_sft= [reward_pipe(i, **sent_kwargs) for i in result_sft]
score_response_ppo= [reward_pipe(i, **sent_kwargs) for i in result_ppo]

for i in range(len(result_ppo)):
    if score_response_ppo[i] > score_response_sft[i]: 
        like.append(1)
    else:
        like.append(0)

print(f'Winner rate of ppo model: {sum(like)/ len(result_ppo)}')

Winner rate of ppo model: 0.72
